In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as MSE
import seaborn as sns
import xgboost
from pandas import read_csv
import pandas
import numpy as np
import matplotlib.pyplot as plt
import csv
from alys import save as SV
from sklearn.model_selection import train_test_split

In [ ]:
df_new = read_csv("new_Basetrain.csv", index_col =0)
df_kagg = read_csv("new_kaggtest.csv", index_col =0)

In [ ]:
y_trainfull = df_new.SalePriceLog.values.reshape((1453,1))
x_trainfull = df_new.drop(['target','SalePriceLog'], axis=1)
x_test = df_kagg

In [ ]:
y_trainfull.shape,x_trainfull.shape,x_test.shape

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_trainfull,y_trainfull , test_size=0.3, random_state=42)

In [ ]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape 

In [ ]:
xgb = xgboost.XGBRegressor(colsample_bytree=0.5, subsample=0.7,
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.8, n_estimators=1000,
                             reg_alpha=0.1, reg_lambda=0.3, gamma=0.01, 
                             silent=1, random_state =7, nthread = -1)
xgb.fit(x_train,y_train)

In [ ]:
#Available importance_types = [‘weight’, ‘gain’, ‘cover’, ‘total_gain’, ‘total_cover’]
f = 'gain'
fi_mod = xgb.get_booster().get_score(importance_type= f)
print(fi_mod)

In [ ]:
fidf_mod = pandas.DataFrame.from_dict(fi_mod, orient='index',columns=['importance_values'])
selec = fidf_mod['importance_values'].sort_values(ascending=False)[0:30]
selec

In [ ]:
plt.figure(figsize=(10,10))
ax = sns.barplot(x=selec ,y=selec.index,)

In [ ]:
xgb_pred = xgb.predict(x_val)

In [ ]:
RMSE = np.sqrt(MSE(xgb_pred, y_val))
print(RMSE.round(4))     

In [ ]:
y_test = xgb.predict(x_test)

In [ ]:
submission = pd.DataFrame({"Id": df_kagg.index,"SalePrice": np.exp(y_test)})
submission.to_csv("new_kagg10.csv", index= False)